# DS Automation Assignment

Using our prepared churn data from week 2:
- use pycaret to find an ML algorithm that performs best on the data
    - Choose a metric you think is best to use for finding the best model; by default, it is accuracy but it could be AUC, precision, recall, etc. The week 3 FTE has some information on these different metrics.
- save the model to disk
- create a Python script/file/module with a function that takes a pandas dataframe as an input and returns the probability of churn for each row in the dataframe
    - your Python file/function should print out the predictions for new data (new_churn_data.csv)
    - the true values for the new data are [1, 0, 0, 1, 0] if you're interested
- test your Python module and function with the new data, new_churn_data.csv
- write a short summary of the process and results at the end of this notebook
- upload this Jupyter Notebook and Python file to a Github repository, and turn in a link to the repository in the week 5 assignment dropbox

*Optional* challenges:
- return the probability of churn for each new prediction, and the percentile where that prediction is in the distribution of probability predictions from the training dataset (e.g. a high probability of churn like 0.78 might be at the 90th percentile)
- use other autoML packages, such as TPOT, H2O, MLBox, etc, and compare performance and features with pycaret
- create a class in your Python module to hold the functions that you created
- accept user input to specify a file using a tool such as Python's `input()` function, the `click` package for command-line arguments, or a GUI
- Use the unmodified churn data (new_unmodified_churn_data.csv) in your Python script. This will require adding the same preprocessing steps from week 2 since this data is like the original unmodified dataset from week 1.

In [1]:
import pandas as pd
from pycaret.classification import setup, compare_models, predict_model, save_model, load_model, models, create_model

In [2]:
# load prepared data
df = pd.read_csv('data/prepped_churn_data.csv', index_col='customerID')
df

,tenure,MonthlyCharges,TotalCharges,PhoneService,Contract,PaymentMethod,Churn,totalcharges_tenure_ratio,monthlycharges_tenure_ratio
customerID,,,,,,,,,
7590-VHVEG,1,29.85,29.85,0,0,2,0,29.850000,29.850000
5575-GNVDE,34,56.95,1889.50,1,1,3,0,55.573529,1.675000
3668-QPYBK,2,53.85,108.15,1,0,3,1,54.075000,26.925000
7795-CFOCW,45,42.30,1840.75,0,1,0,0,40.905556,0.940000
9237-HQITU,2,70.70,151.65,1,0,2,1,75.825000,35.350000
...,...,...,...,...,...,...,...,...,...
6840-RESVB,24,84.80,1990.50,1,1,3,0,82.937500,3.533333
2234-XADUH,72,103.20,7362.90,1,1,1,0,102.262500,1.433333
4801-JZAZL,11,29.60,346.45,0,0,2,0,31.495455,2.690909


In [3]:
# drop these columns just for this exercise (because new_churn_data.csv does not have these features)
df.drop(['totalcharges_tenure_ratio', 'monthlycharges_tenure_ratio'], axis = 1, inplace=True)

# use pycaret to find an ML algorithm that performs best on the data

In [4]:
automl = setup(df, target='Churn') #, use_gpu=True)

,Description,Value
0,session_id,4529
1,Target,Churn
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(7032, 7)"
5,Missing Values,False
6,Numeric Features,3
7,Categorical Features,3
8,Ordinal Features,False
9,High Cardinality Features,False


In [5]:
#automl[12]

To get xgboost and lightgbm working, either allow preprocessing (which converts categorical columns into numeric columns) or set categorical columns as numeric with
```
automl = setup(df, target='Diabetes', preprocess=False, numeric_features=['Gender'])
```
Optionally set an argument sort in compare_models to choose another metric as our scoring metric. By default, it uses accuracy but could set this to `sort='Precision'` to use precision (TP / (TP + FN)).

In [6]:
best_model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.7982,0.8379,0.5181,0.6467,0.5745,0.4445,0.4497,0.2330
lda,Linear Discriminant Analysis,0.7958,0.8284,0.5313,0.6351,0.5780,0.4448,0.4482,0.0050
ada,Ada Boost Classifier,0.7956,0.8394,0.4996,0.6459,0.5622,0.4319,0.4387,0.0170
ridge,Ridge Classifier,0.7948,0.0000,0.4657,0.6570,0.5436,0.4165,0.4275,0.0040
gbc,Gradient Boosting Classifier,0.7948,0.8404,0.5043,0.6407,0.5638,0.4322,0.4378,0.0400
catboost,CatBoost Classifier,0.7903,0.8389,0.5073,0.6264,0.5597,0.4243,0.4289,0.3660
lightgbm,Light Gradient Boosting Machine,0.7859,0.8293,0.5251,0.6090,0.5632,0.4225,0.4250,0.0120
xgboost,Extreme Gradient Boosting,0.7790,0.8185,0.5158,0.5924,0.5509,0.4053,0.4074,0.1110
knn,K Neighbors Classifier,0.7741,0.7527,0.4594,0.5926,0.5166,0.3724,0.3782,0.1980
rf,Random Forest Classifier,0.7629,0.7953,0.4757,0.5591,0.5135,0.3582,0.3606,0.0380


In [7]:
best_model

# models()  # to obtain list of id's available in pycaret
# ada = create_model('ada')  # use create_model()

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=4529, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [8]:
# select the last row of data but make it a 2D array [-2:-1]
df.iloc[-2:-1].shape

(1, 7)

`df.iloc[-1].shape` only works if `preprocess=False` in pycaret setup() function, otherwise the order of features may be different.

In [9]:
#predict_model(best_model, df.iloc[-2:-1])
predict_model(best_model, df)

,tenure,MonthlyCharges,TotalCharges,PhoneService,Contract,PaymentMethod,Churn,Label,Score
customerID,,,,,,,,,
7590-VHVEG,1,29.85,29.85,0,0,2,0,1,0.5484
5575-GNVDE,34,56.95,1889.50,1,1,3,0,0,0.9334
3668-QPYBK,2,53.85,108.15,1,0,3,1,0,0.5927
7795-CFOCW,45,42.30,1840.75,0,1,0,0,0,0.9583
9237-HQITU,2,70.70,151.65,1,0,2,1,1,0.6425
...,...,...,...,...,...,...,...,...,...
6840-RESVB,24,84.80,1990.50,1,1,3,0,0,0.7877
2234-XADUH,72,103.20,7362.90,1,1,1,0,0,0.9090
4801-JZAZL,11,29.60,346.45,0,0,2,0,0,0.5806


In [10]:
# Assignment: save the model to disk
save_model(best_model, 'LR')

Transformation Pipeline and Model Succesfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[],
                                       ml_usecase='classification',
                                       numerical_features=[], target='Churn',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numeric_strate...
                 ('feature_select', 'passthrough'), ('fix_multi', 'passthrough'),
                 ('dfs', 'passthrough'), ('pca', 'passthrough'),
                 ['trained_model',
                  LogisticRegression(C=1.0, class_weight=None, dual=False,
                 

In [11]:
loaded_lr = load_model('LR')

Transformation Pipeline and Model Successfully Loaded


In [12]:
new_data = df.iloc[-2:-1].copy()
new_data.drop(['Churn'], axis=1, inplace=True)
new_data

,tenure,MonthlyCharges,TotalCharges,PhoneService,Contract,PaymentMethod
customerID,,,,,,
8361-LTMKD,4,74.4,306.6,1,0,3


In [13]:
predict_model(loaded_lr, data=new_data)

,tenure,MonthlyCharges,TotalCharges,PhoneService,Contract,PaymentMethod,Label,Score
customerID,,,,,,,,
8361-LTMKD,4,74.4,306.6,1,0,3,1,0.5197


In [14]:
from IPython.display import Code

Code('predict_churn.py')

import os
import click
import pandas as pd
from pycaret.classification import predict_model, load_model


# OPTIONAL: create a class in your Python module to hold the functions that you created
class ChurnPredictor:
    def load_data(filepath):
        """
        Loads data into a DataFrame from a string filepath
        """
        df = pd.read_csv(filepath, index_col='customerID')

        return df


    def make_predictions(dataframe):
        """
        Uses the pycaret best model to make predictions on data in the df dataframe
        """
        model = load_model('LR')
        predictions = predict_model(model, data=dataframe)
        predictions.rename({'Label': 'Churn_prediction'}, axis=1, inplace=True)
        predictions['Churn_prediction'].replace({1: 'Churn', 0: 'No churn'},
                                                inplace=True)
        
        return predictions['Churn_prediction']


# OPTIONAL: accept user input to specify a file using a tool such as
#           i) Python's input() function, 
#           ii) the ***click*** package for command-line arguments, or
#           iii) GUI
@click.command()
@click.option("--filename", 
              prompt="Enter the full path to a CSV file", 
              help="ex. data\\new_churn_data.csv",
              default='data\\new_churn_data.csv')
def prompt_for_file(filename):
    try:
        if os.path.exists(filename):
            df = ChurnPredictor.load_data(filename)    # ('data/new_churn_data.csv')
            predictions = ChurnPredictor.make_predictions(df)
            print('Predictions:\n{}'.format(predictions))
        else:
            print('Unable to locate file {}'.format(str(filename)))
    except Exception as error:
        print('Error occured: {}'.format(str(error)))


if __name__ == "__main__":
    # prompt for filename
    # bool_return = prompt_for_file()

    df = ChurnPredictor.load_data('data/new_churn_data.csv')
    predictions = ChurnPredictor.make_predictions(df)
    print('predictions:')
    print(predictions)

In [15]:
%run predict_churn.py

Transformation Pipeline and Model Successfully Loaded
predictions:
customerID
9305-CKSKC       Churn
1452-KNGVK    No churn
6723-OKKJM    No churn
7832-POPKP    No churn
6348-TACGU       Churn
Name: Churn_prediction, dtype: object


# Summary

## Write a short summary of the process and results here

- I loaded the CSV into a Pandas Dataframe
- I setup the pycaret.classification module
- I used compare_models() method to find the best model based on Accuracy
- With the best_model recommendation, I saved the Model to disk in a Pickle file
- I created a Python script to read new data and make predictions
- The predictions of Churn on the 5 new Customers was 1, 0, 0, 0, 0


## Upload to Github

I uploaded this notebook (Week_5_assignment_starter.ipynb) and the Python prediction script (predict_churn.py) to Github.

# Optional

use other autoML packages, such as TPOT, H2O, MLBox, etc, and compare performance and features with pycaret

H2O.ai AutoML - https://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.html

In [16]:
import h2o
from h2o.automl import H2OAutoML

# Start the H2O cluster (locally)
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.291-b10, mixed mode)
  Starting server from C:\ProgramData\Anaconda3\envs\msds600\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\marty\AppData\Local\Temp\tmpe3vzu2xo
  JVM stdout: C:\Users\marty\AppData\Local\Temp\tmpe3vzu2xo\h2o_marty_started_from_python.out
  JVM stderr: C:\Users\marty\AppData\Local\Temp\tmpe3vzu2xo\h2o_marty_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,America/Chicago
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.0.2
H2O_cluster_version_age:,19 days
H2O_cluster_name:,H2O_from_python_marty_ofcogi
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,14.21 Gb
H2O_cluster_total_cores:,24
H2O_cluster_allowed_cores:,24
H2O_cluster_status:,"locked, healthy"


In [17]:
import pandas as pd
hf = h2o.H2OFrame(pd.read_csv('data/prepped_churn_data.csv', index_col='customerID'))

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [18]:
hf

tenure,MonthlyCharges,TotalCharges,PhoneService,Contract,PaymentMethod,Churn,totalcharges_tenure_ratio,monthlycharges_tenure_ratio
1,29.85,29.85,0,0,2,0,29.85,29.85
34,56.95,1889.5,1,1,3,0,55.5735,1.675
2,53.85,108.15,1,0,3,1,54.075,26.925
45,42.3,1840.75,0,1,0,0,40.9056,0.94
2,70.7,151.65,1,0,2,1,75.825,35.35
8,99.65,820.5,1,0,2,1,102.562,12.4563
22,89.1,1949.4,1,0,1,0,88.6091,4.05
10,29.75,301.9,0,0,3,0,30.19,2.975
28,104.8,3046.05,1,0,2,1,108.787,3.74286
62,56.15,3487.95,1,1,0,0,56.2573,0.905645


In [19]:
# break into train and test/valid sets
# set the predictors and response
hf["Churn"] = hf["Churn"].asfactor()    # for binary classification, response should be a factor - https://docs.h2o.ai/h2o/latest-stable/h2o-docs/grid-search.html
predictors = hf.columns
predictors.remove('Churn')
response = "Churn"

# Split the dataset into a train and valid set:
train, valid = hf.split_frame(ratios=[.8], seed=1234, )

aml = H2OAutoML(max_models=20, seed=1)
aml.train(x=predictors,
          y=response,
          training_frame=train)

# Note: XGBoost is not available on Windows machines.
# https://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.html

AutoML progress: |
19:00:40.484: AutoML: XGBoost is not available; skipping it.
19:00:40.495: Step 'best_of_family_xgboost' not defined in provider 'StackedEnsemble': skipping it.
19:00:40.496: Step 'all_xgboost' not defined in provider 'StackedEnsemble': skipping it.

███████████████████████████████████████████████████████████████| (done) 100%
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_7_AutoML_1_20220214_190040

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.12673337449114266
RMSE: 0.35599631246846175
LogLoss: 0.39106717563885873
Null degrees of freedom: 5654
Residual degrees of freedom: 5650
Null deviance: 6582.989014751938
Residual deviance: 4422.969756475494
AIC: 4432.969756475494
AUC: 0.8717195952396105
AUCPR: 0.708246886690085
Gini: 0.743439190479221

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3492035493345102: 


,,0,1,Error,Rate
0,0,3307.0,828.0,0.2002,(828.0/4135.0)
1,1,330.0,1190.0,0.2171,(330.0/1520.0)
2,Total,3637.0,2018.0,0.2048,(1158.0/5655.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.349204,0.672696,201.0
1,max f2,0.210172,0.775133,271.0
2,max f0point5,0.519768,0.674697,125.0
3,max accuracy,0.519768,0.819275,125.0
4,max precision,0.877005,1.000000,0.0
5,max recall,0.001035,1.000000,398.0
6,max specificity,0.877005,1.000000,0.0
7,max absolute_mcc,0.349204,0.539184,201.0
8,max min_per_class_accuracy,0.342650,0.790326,205.0
9,max mean_per_class_accuracy,0.345661,0.791332,203.0



Gains/Lift Table: Avg response rate: 26.88 %, avg score: 27.61 %


...

,,0,1,Error,Rate
0,0,3365.0,770.0,0.1862,(770.0/4135.0)
1,1,450.0,1070.0,0.2961,(450.0/1520.0)
2,Total,3815.0,1840.0,0.2157,(1220.0/5655.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.356038,0.636905,194.0
1,max f2,0.162858,0.753120,291.0
2,max f0point5,0.508074,0.624147,128.0
3,max accuracy,0.508074,0.798055,128.0
4,max precision,0.882318,0.909091,2.0
5,max recall,0.004547,1.000000,397.0
6,max specificity,0.902745,0.999758,0.0
7,max absolute_mcc,0.358692,0.490120,193.0
8,max min_per_class_accuracy,0.308770,0.762999,216.0
9,max mean_per_class_accuracy,0.250348,0.767756,246.0



Gains/Lift Table: Avg response rate: 26.88 %, avg score: 26.89 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010080,0.841543,3.198234,3.198234,0.859649,0.863536,0.859649,0.863536,0.032237,0.032237,219.823407,219.823407,0.030302
1,2,0.020159,0.804129,3.002424,3.100329,0.807018,0.821411,0.833333,0.842474,0.030263,0.062500,200.242382,210.032895,0.057905
2,3,0.030062,0.785616,2.923167,3.041970,0.785714,0.794178,0.817647,0.826565,0.028947,0.091447,192.316729,204.196981,0.083950
3,4,0.040141,0.764867,3.002424,3.032040,0.807018,0.776095,0.814978,0.813892,0.030263,0.121711,200.242382,203.203976,0.111553
4,5,0.050044,0.734677,2.723860,2.971057,0.732143,0.749152,0.798587,0.801081,0.026974,0.148684,172.386043,197.105728,0.134899
5,6,0.100088,0.641896,2.734424,2.852741,0.734982,0.684051,0.766784,0.742566,0.136842,0.285526,173.442440,185.274084,0.253604
6,7,0.150133,0.557211,2.221720,2.642401,0.597173,0.598099,0.710247,0.694410,0.111184,0.396711,122.171983,164.240050,0.337218
7,8,0.200000,0.498781,1.899776,2.457237,0.510638,0.526545,0.660477,0.652556,0.094737,0.491447,89.977604,145.723684,0.398582
8,9,0.300088,0.382420,1.761600,2.225221,0.473498,0.446057,0.598114,0.583682,0.176316,0.667763,76.160033,122.522137,0.502830
9,10,0.400000,0.288114,1.152335,1.957237,0.309735,0.333752,0.526083,0.521255,0.115132,0.782895,15.233465,95.723684,0.523644


In [20]:
# View the AutoML Leaderboard (comparable to pycaret's compare_models())
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_BestOfFamily_7_AutoML_1_20220214_190040,0.844429,0.419266,0.650606,0.241134,0.369815,0.136763
StackedEnsemble_BestOfFamily_4_AutoML_1_20220214_190040,0.844307,0.419093,0.648308,0.240665,0.369971,0.136879
StackedEnsemble_AllModels_4_AutoML_1_20220214_190040,0.844147,0.41914,0.650516,0.241283,0.369984,0.136888
StackedEnsemble_BestOfFamily_5_AutoML_1_20220214_190040,0.843997,0.41956,0.64986,0.241932,0.370075,0.136955
StackedEnsemble_BestOfFamily_2_AutoML_1_20220214_190040,0.843924,0.419516,0.650569,0.243486,0.370073,0.136954
StackedEnsemble_BestOfFamily_1_AutoML_1_20220214_190040,0.843865,0.419633,0.648819,0.242131,0.370095,0.13697
StackedEnsemble_BestOfFamily_3_AutoML_1_20220214_190040,0.843814,0.419696,0.64818,0.241923,0.370183,0.137036
StackedEnsemble_AllModels_2_AutoML_1_20220214_190040,0.84373,0.419778,0.650035,0.242494,0.370081,0.13696
StackedEnsemble_AllModels_1_AutoML_1_20220214_190040,0.843558,0.419931,0.648165,0.241086,0.37031,0.13713
StackedEnsemble_AllModels_6_AutoML_1_20220214_190040,0.843379,0.421233,0.650142,0.238152,0.370985,0.13763


In [21]:
# leader model (comparable to pycaret's best_model)
aml.leader

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_7_AutoML_1_20220214_190040

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.12673337449114266
RMSE: 0.35599631246846175
LogLoss: 0.39106717563885873
Null degrees of freedom: 5654
Residual degrees of freedom: 5650
Null deviance: 6582.989014751938
Residual deviance: 4422.969756475494
AIC: 4432.969756475494
AUC: 0.8717195952396105
AUCPR: 0.708246886690085
Gini: 0.743439190479221

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3492035493345102: 


,,0,1,Error,Rate
0,0,3307.0,828.0,0.2002,(828.0/4135.0)
1,1,330.0,1190.0,0.2171,(330.0/1520.0)
2,Total,3637.0,2018.0,0.2048,(1158.0/5655.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.349204,0.672696,201.0
1,max f2,0.210172,0.775133,271.0
2,max f0point5,0.519768,0.674697,125.0
3,max accuracy,0.519768,0.819275,125.0
4,max precision,0.877005,1.000000,0.0
5,max recall,0.001035,1.000000,398.0
6,max specificity,0.877005,1.000000,0.0
7,max absolute_mcc,0.349204,0.539184,201.0
8,max min_per_class_accuracy,0.342650,0.790326,205.0
9,max mean_per_class_accuracy,0.345661,0.791332,203.0



Gains/Lift Table: Avg response rate: 26.88 %, avg score: 27.61 %


...

,,0,1,Error,Rate
0,0,3365.0,770.0,0.1862,(770.0/4135.0)
1,1,450.0,1070.0,0.2961,(450.0/1520.0)
2,Total,3815.0,1840.0,0.2157,(1220.0/5655.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.356038,0.636905,194.0
1,max f2,0.162858,0.753120,291.0
2,max f0point5,0.508074,0.624147,128.0
3,max accuracy,0.508074,0.798055,128.0
4,max precision,0.882318,0.909091,2.0
5,max recall,0.004547,1.000000,397.0
6,max specificity,0.902745,0.999758,0.0
7,max absolute_mcc,0.358692,0.490120,193.0
8,max min_per_class_accuracy,0.308770,0.762999,216.0
9,max mean_per_class_accuracy,0.250348,0.767756,246.0



Gains/Lift Table: Avg response rate: 26.88 %, avg score: 26.89 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010080,0.841543,3.198234,3.198234,0.859649,0.863536,0.859649,0.863536,0.032237,0.032237,219.823407,219.823407,0.030302
1,2,0.020159,0.804129,3.002424,3.100329,0.807018,0.821411,0.833333,0.842474,0.030263,0.062500,200.242382,210.032895,0.057905
2,3,0.030062,0.785616,2.923167,3.041970,0.785714,0.794178,0.817647,0.826565,0.028947,0.091447,192.316729,204.196981,0.083950
3,4,0.040141,0.764867,3.002424,3.032040,0.807018,0.776095,0.814978,0.813892,0.030263,0.121711,200.242382,203.203976,0.111553
4,5,0.050044,0.734677,2.723860,2.971057,0.732143,0.749152,0.798587,0.801081,0.026974,0.148684,172.386043,197.105728,0.134899
5,6,0.100088,0.641896,2.734424,2.852741,0.734982,0.684051,0.766784,0.742566,0.136842,0.285526,173.442440,185.274084,0.253604
6,7,0.150133,0.557211,2.221720,2.642401,0.597173,0.598099,0.710247,0.694410,0.111184,0.396711,122.171983,164.240050,0.337218
7,8,0.200000,0.498781,1.899776,2.457237,0.510638,0.526545,0.660477,0.652556,0.094737,0.491447,89.977604,145.723684,0.398582
8,9,0.300088,0.382420,1.761600,2.225221,0.473498,0.446057,0.598114,0.583682,0.176316,0.667763,76.160033,122.522137,0.502830
9,10,0.400000,0.288114,1.152335,1.957237,0.309735,0.333752,0.526083,0.521255,0.115132,0.782895,15.233465,95.723684,0.523644


In [22]:
# make predictions directly on the `H2OAutoML` object
#predictions = aml.predict(valid)

# or on the leader model
predictions = aml.leader.predict(valid)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


### Predictions on prepped_churn_data.csv

In [23]:
print(predictions)

predict,p0,p1
0,0.979955,0.0200452
0,0.96817,0.0318302
1,0.342771,0.657229
0,0.92773,0.0722697
0,0.735175,0.264825
1,0.462905,0.537095
1,0.262313,0.737687
0,0.911973,0.088027
1,0.509368,0.490632
0,0.756082,0.243918


**Note: Cannot save AutoML Models**

Rather than saving an AutoML object itself, currently, the best thing to do is to save the models you want to keep, individually. A utility for saving all of the models at once, along with a way to save the AutoML object (with leaderboard), will be added in a future release.

https://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.html

## Predictions on new_churn_data.csv

In [26]:
hf = h2o.H2OFrame(pd.read_csv('data/new_churn_data.csv', index_col='customerID'))

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [28]:
new_predictions = aml.leader.predict(hf)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


In [29]:
print(new_predictions)

predict,p0,p1
1,0.41551,0.58449
0,0.69997,0.30003
0,0.820066,0.179934
1,0.6349,0.3651
1,0.562583,0.437417
